# 

For this notebook we use a 4CAT corpus collected from TikTok about the
[2024 Farmers’ Protest in
Germany](https://de.wikipedia.org/wiki/Bauernproteste_in_Deutschland_ab_Dezember_2023).
Let’s take a look at all relevant columns. We’re mostly dealing with the
`image_file` column. Additionally, the images files should be extracted
to the `/content/media/images/` path. (See linked notebook for the
conversion from the original 4CAT files).

In [77]:
df[['id', 'body', 'Transcript', 'image_file']].head()

## BERTopic

Let’s first install `bertopic` including the vision extensions.

> **Note**
>
> The following code has been taken from the [BERTopic
> documentation](https://maartengr.github.io/BERTopic/getting_started/multimodal/multimodal.html)
> and was only slightly changed.

In [ ]:
!pip install bertopic[vision]

### Images Only

Next, we prepare the pipeline for an image-only model: We want to fit
the Topic Model on the image content only. We follow [the BERTOpic
Multimodal
Manual](https://maartengr.github.io/BERTopic/getting_started/multimodal/multimodal.html#images-only),
and generate image captions using the
`vit-gpt2-image-captioning`package. The documentation offers a lot of
different options, we can incorporate textual content for the topic
modeling, or fit the model on textual information only and look for the
best matching images for each cluster and display them.

In our example we focus on image-only topics models.

In [ ]:
from bertopic.representation import KeyBERTInspired, VisualRepresentation
from bertopic.backend import MultiModalBackend

# Image embedding model
embedding_model = MultiModalBackend('clip-ViT-B-32', batch_size=32)

# Image to text representation model
representation_model = {
    "Visual_Aspect": VisualRepresentation(image_to_text_model="nlpconnect/vit-gpt2-image-captioning")
}

Next, select the column with the path of your images files, in my
example `image_file`. Convert it to a python `list`.

In [74]:
image_only_df = df.copy()
images = image_only_df['image_file'].to_list()

Now it’s time to fit the model.

In [75]:
from bertopic import BERTopic

# Train our model with images only
topic_model = BERTopic(embedding_model=embedding_model, representation_model=representation_model, min_topic_size=5)
topics, probs = topic_model.fit_transform(documents=None, images=images)

100%|██████████| 7/7 [00:02<00:00,  2.99it/s]

Finally let’s display the topics. **Remember:** Topic `-1` is a
collection of documenst that do not fit into any topic.

In [ ]:
# See linked notebook for code.